# Библиотеки

In [112]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import re
from collections import Counter
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import numpy as np

# Обзор данных 

In [113]:
data = pd.read_csv("vacancy_list_test.csv")
resume_data = pd.read_csv("resume_list_test.csv")

In [114]:
resume_data.info()
resume_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3304 entries, 0 to 3303
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      3304 non-null   int64 
 1   title           3304 non-null   object
 2   salary          3304 non-null   object
 3   experience      3304 non-null   object
 4   gender          3304 non-null   object
 5   age             3304 non-null   object
 6   city            3304 non-null   object
 7   tags_list       3304 non-null   object
 8   tags_list_len   3304 non-null   int64 
 9   skills-content  3064 non-null   object
 10  language        3304 non-null   object
 11  work_text       3301 non-null   object
 12  specialization  3288 non-null   object
dtypes: int64(2), object(11)
memory usage: 335.7+ KB


,Unnamed: 0,title,salary,experience,gender,age,city,tags_list,tags_list_len,skills-content,language,work_text,specialization
0,0,"CMO, Head of marketing/digital marketing, дире...",200 000 руб.,Опыт работы 10 лет 10 месяцев,Мужчина,29 лет,Санкт-Петербург,"['Adobe Illustrator', 'Adobe InDesign', 'Adobe...",26,Искренне люблю удобные цифровые проекты и клие...,"['Русский — Родной', 'Английский — B1 — Средний']",Онлайн сервис для студентов\nДиректор по марке...,Специализации:\nРуководитель отдела маркетинга...
1,1,Директор по маркетингу e-commerce,350 000 руб.,Опыт работы 11 лет 8 месяцев,Мужчина,33 года,Санкт-Петербург,"['E-Commerce', 'Internet Marketing', 'Ведение ...",15,В e-commerce прошел путь от менеджера интернет...,"['Русский — Родной', 'Английский — C1 — Продви...",Kotofoto.ru\nДиректор по маркетингу и электрон...,Специализации:\nДиректор по маркетингу и PR (CMO)
2,2,Директор по маркетингу,200 000 руб.,Опыт работы 17 лет 6 месяцев,Женщина,38 лет,Санкт-Петербург,"['B2B маркетинг', 'Internet Marketing', 'Бюдже...",17,1. 15 лет опыта в маркетинге: B2B & B2C.\n2. В...,"['Русский — Родной', 'Английский — C1 — Продви...","Ренталформ\nСанкт-Петербург, rentalform.ru/\nД...",Специализации:\nEvent-менеджер\nМенеджер по ма...
3,3,Директор по маркетингу,185 000 руб.,Опыт работы 17 лет 6 месяцев,Женщина,39 лет,Санкт-Петербург,"['Маркетинговые коммуникации', 'Бизнес-планиро...",22,"Увлечения, хобби: театральное и изобразительно...","['Русский — Родной', 'Английский — C1 — Продви...","Рыбаков Плейскул (экосистема Игоря Рыбакова, м...",Специализации:\nДиректор по маркетингу и PR (C...
4,4,Руководитель отдела маркетинга,150 000 руб.,Опыт работы 6 лет 11 месяцев,Женщина,40 лет,Москва,['Обучаемость'],1,MS Excel\nГрафический дизайн (CorelDRAW)\nКреа...,"['Русский — Родной', 'Английский — B2 — Средне...","ГК ""Спецпродукт""\nМосква, savia.ru\nРуководите...",Специализации:\nДиректор по маркетингу и PR (C...


In [115]:
resume_data.dtypes

Unnamed: 0         int64
title             object
salary            object
experience        object
gender            object
age               object
city              object
tags_list         object
tags_list_len      int64
skills-content    object
language          object
work_text         object
specialization    object
dtype: object

In [116]:
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1709 entries, 0 to 1708
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1709 non-null   int64 
 1   city2          906 non-null    object
 2   experience     1709 non-null   object
 3   description    1709 non-null   object
 4   salary         1709 non-null   object
 5   tags_list      1709 non-null   object
 6   tags_list_len  1709 non-null   int64 
 7   city           803 non-null    object
dtypes: int64(2), object(6)
memory usage: 106.9+ KB


,Unnamed: 0,city2,experience,description,salary,tags_list,tags_list_len,city
0,0,"Улица 1905 года, Москва, улица 1905 года, 10с1",1–3 года,Привет наш будущий кандидат на должность марке...,от 120 000 руб. на руки,"['SWOT анализ', 'Adobe Photoshop', 'Работа в к...",5,NaN
1,1,NaN,3–6 лет,Привет! Мы Семь Семян - онлайн супермаркет тов...,от 200 000 руб. на руки,"['Медиапланирование', 'Интернет-реклама', 'Кон...",5,"Москва, Румянцево"
2,2,"Электросталь, Строительный переулок, 10",более 6 лет,LEGRAND - производитель федерального уровня с ...,от 200 000 до 400 000 руб. на руки,"['B2B маркетинг', 'unit-экономика', 'Запуск но...",12,NaN
3,3,NaN,1–3 года,Наша школа Футбола и спортивной психологии явл...,от 60 000 до 120 000 руб. на руки,"['E-Mail Marketing', 'Стратегический маркетинг...",9,Москва
4,4,NaN,более 6 лет,Мы - международная FinTech компания AMarkets. ...,от 10 000 USD на руки,[],0,Москва


In [117]:
data.dtypes

Unnamed: 0        int64
city2            object
experience       object
description      object
salary           object
tags_list        object
tags_list_len     int64
city             object
dtype: object

In [118]:
data.isna().sum()

Unnamed: 0         0
city2            803
experience         0
description        0
salary             0
tags_list          0
tags_list_len      0
city             906
dtype: int64

In [119]:
resume_data.isna().sum()

Unnamed: 0          0
title               0
salary              0
experience          0
gender              0
age                 0
city                0
tags_list           0
tags_list_len       0
skills-content    240
language            0
work_text           3
specialization     16
dtype: int64

In [120]:
print(f"Дубликаты: {resume_data.duplicated().sum()}")
print(f"Дубликаты: {data.duplicated().sum()}")

Дубликаты: 0
Дубликаты: 0


Теперь в tags_list списки из строк, вместо строк

In [121]:
def str_to_list(str_repr):
  if str_repr == " ":
    return []
  else:
    return [s.strip().strip("'") for s in str_repr[1:-1].split(",")]
data["tags_list"] = [str_to_list(i) for i in data["tags_list"]]

In [122]:
data.isnull().sum()

Unnamed: 0         0
city2            803
experience         0
description        0
salary             0
tags_list          0
tags_list_len      0
city             906
dtype: int64

In [123]:
data["city"].duplicated().sum()

np.int64(1643)

In [124]:
data.fillna("", inplace=True)

In [125]:
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1709 entries, 0 to 1708
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1709 non-null   int64 
 1   city2          1709 non-null   object
 2   experience     1709 non-null   object
 3   description    1709 non-null   object
 4   salary         1709 non-null   object
 5   tags_list      1709 non-null   object
 6   tags_list_len  1709 non-null   int64 
 7   city           1709 non-null   object
dtypes: int64(2), object(6)
memory usage: 106.9+ KB


,Unnamed: 0,city2,experience,description,salary,tags_list,tags_list_len,city
0,0,"Улица 1905 года, Москва, улица 1905 года, 10с1",1–3 года,Привет наш будущий кандидат на должность марке...,от 120 000 руб. на руки,"[SWOT анализ, Adobe Photoshop, Работа в команд...",5,
1,1,,3–6 лет,Привет! Мы Семь Семян - онлайн супермаркет тов...,от 200 000 руб. на руки,"[Медиапланирование, Интернет-реклама, Контекст...",5,"Москва, Румянцево"
2,2,"Электросталь, Строительный переулок, 10",более 6 лет,LEGRAND - производитель федерального уровня с ...,от 200 000 до 400 000 руб. на руки,"[B2B маркетинг, unit-экономика, Запуск новых п...",12,
3,3,,1–3 года,Наша школа Футбола и спортивной психологии явл...,от 60 000 до 120 000 руб. на руки,"[E-Mail Marketing, Стратегический маркетинг, А...",9,Москва
4,4,,более 6 лет,Мы - международная FinTech компания AMarkets. ...,от 10 000 USD на руки,[],0,Москва


Интересно узнать какой опыт работы интересует работадателей

In [126]:
pd.DataFrame(data["experience"].unique(), columns=["experience"])

,experience
0,1–3 года
1,3–6 лет
2,более 6 лет
3,не требуется


In [127]:
data_tags = set()
s=0
for i in data["tags_list"]:
    for j in i:
        data_tags.add(j)
data_tags

{'',
 'Программное обеспечение',
 'Дизайн-мышление',
 'Протоколирование',
 'Тестирование',
 'Индивидуальное обучение',
 'ассистент',
 'Управление лояльностью потребителей',
 'Дизайн интерфейсов',
 'Оперативный поиск информации в сети Интернет',
 'Jenkins',
 'Ответственность и пунктуальность',
 'Автоматизация продаж',
 'Customer Journey Map',
 'B2B Marketing',
 'Наставничество',
 'Key Collector',
 'Коммуникативная стратегия',
 'Высокая ответственность',
 'Коучинг',
 'Snapchat',
 'Наружная реклама',
 'Проведение маркетинговых исследований',
 'продажа недвижимости',
 'Прием посетителей',
 'Торговля',
 'Сопровождение сделок',
 'Активная жизненная позиция',
 'Поддержка работы офиса',
 'Разработка продукта',
 'Релокация иностранных сотрудников',
 'Онлайн школа',
 'Управленческая отчетность',
 'HR',
 'Исследования рынка',
 'Автоматизация',
 'Битрикс-24',
 'Открытие и закрытие счетов',
 'Спорт',
 'продажи услуг',
 '1C: Бухгалтерия',
 'Описание бизнес-процессов',
 'Адаптация рекламных роликов',

# Предобработка данных

В таблицах data и resume_data неудобный формат столбцов salary. Разделим salary на 3 отдельных столбца: from, upto и currency.
* from - минимальная зарплата
* upto - максимальная зарплата
* currency - валюта
  
В resume_data везде указана одна зарплата вместо зарплатной вилки. Поэтому разделим salary на 2 отдельных столбца: salary и currency.
* salary - зарплата
* currency - валюта

#### Data. Создаем from и upto

In [128]:

frac_data = data.sample(frac = 0.1)
salarys = []
for i in data["salary"]:
    salary = re.findall(r'\d+\s\d+', i)
    salarys.append([int(i.replace(" ", "")) for i in salary])
df = pd.DataFrame(salarys)
df = df.fillna(0)

data["from"] = df[0]
data["upto"] = df[1]
data[["from","upto"]]

,from,upto
0,120000.0,0.0
1,200000.0,0.0
2,200000.0,400000.0
3,60000.0,120000.0
4,10000.0,0.0
...,...,...
1704,60000.0,100000.0
1705,100000.0,0.0
1706,100000.0,0.0
1707,100000.0,0.0


#### Data. Создаем currency

In [129]:

curr = []
for i in data["salary"]:
    temp = re.findall(r'руб|USD|EUR|не указана|сум', i)
    curr.append(temp[0])

data["currency"] = curr
data.head(2)

,Unnamed: 0,city2,experience,description,salary,tags_list,tags_list_len,city,from,upto,currency
0,0,"Улица 1905 года, Москва, улица 1905 года, 10с1",1–3 года,Привет наш будущий кандидат на должность марке...,от 120 000 руб. на руки,"[SWOT анализ, Adobe Photoshop, Работа в команд...",5,,120000.0,0.0,руб
1,1,,3–6 лет,Привет! Мы Семь Семян - онлайн супермаркет тов...,от 200 000 руб. на руки,"[Медиапланирование, Интернет-реклама, Контекст...",5,"Москва, Румянцево",200000.0,0.0,руб


Проверяем пересечения city и city2. Так как они отстутствуют можно спокойно их соединить

In [130]:
data[(data["city2"]!="") & (data["city"]!="")]

,Unnamed: 0,city2,experience,description,salary,tags_list,tags_list_len,city,from,upto,currency


Так как пересечения отсутствуют можно спокойно их соединить

In [131]:
data = data.rename(columns={"Unnamed: 0": "index"})
resume_data = resume_data.rename(columns={"Unnamed: 0": "index"})
data["fulladress"] = data["city"] + data["city2"]
data.drop(['city', 'city2'], axis= 1 , inplace= True )
data

,index,experience,description,salary,tags_list,tags_list_len,from,upto,currency,fulladress
0,0,1–3 года,Привет наш будущий кандидат на должность марке...,от 120 000 руб. на руки,"[SWOT анализ, Adobe Photoshop, Работа в команд...",5,120000.0,0.0,руб,"Улица 1905 года, Москва, улица 1905 года, 10с1"
1,1,3–6 лет,Привет! Мы Семь Семян - онлайн супермаркет тов...,от 200 000 руб. на руки,"[Медиапланирование, Интернет-реклама, Контекст...",5,200000.0,0.0,руб,"Москва, Румянцево"
2,2,более 6 лет,LEGRAND - производитель федерального уровня с ...,от 200 000 до 400 000 руб. на руки,"[B2B маркетинг, unit-экономика, Запуск новых п...",12,200000.0,400000.0,руб,"Электросталь, Строительный переулок, 10"
3,3,1–3 года,Наша школа Футбола и спортивной психологии явл...,от 60 000 до 120 000 руб. на руки,"[E-Mail Marketing, Стратегический маркетинг, А...",9,60000.0,120000.0,руб,Москва
4,4,более 6 лет,Мы - международная FinTech компания AMarkets. ...,от 10 000 USD на руки,[],0,10000.0,0.0,USD,Москва
...,...,...,...,...,...,...,...,...,...,...
1704,1704,3–6 лет,Мы ищем человека с активной жизненной позицией...,от 60 000 до 100 000 руб. на руки,"[B2B маркетинг, Анализ бизнес показателей, Ори...",11,60000.0,100000.0,руб,"Пенза, Стрельбищенская улица, 60"
1705,1705,3–6 лет,Российский торгово-производственный холдинг КО...,от 100 000 руб. на руки,"[маркетплейсы, SEO оптимизация, Аналитическое ...",10,100000.0,0.0,руб,"Москва, Ферганская улица, 8А"
1706,1706,1–3 года,Привет!\nМы рады приветствовать тебя на страни...,от 100 000 руб. на руки,"[Управление персоналом, Работа в команде, Анал...",7,100000.0,0.0,руб,Москва
1707,1707,3–6 лет,«Погрызухин» - это корма и лакомства для собак...,от 100 000 руб. на руки,"[Активные продажи, Управление продажами, Разви...",4,100000.0,0.0,руб,"Красногорск, Ильинское шоссе, 1А"


In [132]:
data.head(5)

,index,experience,description,salary,tags_list,tags_list_len,from,upto,currency,fulladress
0,0,1–3 года,Привет наш будущий кандидат на должность марке...,от 120 000 руб. на руки,"[SWOT анализ, Adobe Photoshop, Работа в команд...",5,120000.0,0.0,руб,"Улица 1905 года, Москва, улица 1905 года, 10с1"
1,1,3–6 лет,Привет! Мы Семь Семян - онлайн супермаркет тов...,от 200 000 руб. на руки,"[Медиапланирование, Интернет-реклама, Контекст...",5,200000.0,0.0,руб,"Москва, Румянцево"
2,2,более 6 лет,LEGRAND - производитель федерального уровня с ...,от 200 000 до 400 000 руб. на руки,"[B2B маркетинг, unit-экономика, Запуск новых п...",12,200000.0,400000.0,руб,"Электросталь, Строительный переулок, 10"
3,3,1–3 года,Наша школа Футбола и спортивной психологии явл...,от 60 000 до 120 000 руб. на руки,"[E-Mail Marketing, Стратегический маркетинг, А...",9,60000.0,120000.0,руб,Москва
4,4,более 6 лет,Мы - международная FinTech компания AMarkets. ...,от 10 000 USD на руки,[],0,10000.0,0.0,USD,Москва


#### Рассмотрим from и upto
Так как подавляющее количество данных с currency = руб. То рассмотрим для удобства только эти данные.
Видим, что половина вакансий указывает зарплату от, вторая половина указывает точную зарплатную вилку. Есть аномально большие зарплаты, но они слишком малочисленны, можем предположить о нерентабельности таких данных.

In [133]:
df = data[data["currency"]=="руб"]
df_upto = df["upto"][df["upto"] != 0]
df_from = df["from"][df['from'] != 0]
pd.concat([df_upto, df_from], axis=1).describe()

,upto,from
count,813.000000,1672.000000
mean,161744.650677,113004.729665
std,110354.960137,72081.174962
min,1000.000000,1000.000000
25%,90000.000000,65000.000000
50%,140000.000000,100000.000000
75%,200000.000000,150000.000000
max,800000.000000,700000.000000


Взглянем на распределение возрастов. Заодно проверим нет ли особо древних или несовершеннолетних соискателей

In [134]:
resume_data["age"] = [int(i[:2]) for i in resume_data["age"]]

In [135]:
df = resume_data["age"].groupby(resume_data["age"]).count()
median = resume_data["age"].median()
moda = resume_data["age"].mode()
mean = resume_data["age"].mean()
print(median, moda[0], mean)

fig = go.Figure(layout=dict(barcornerradius=7,))
quartiles = resume_data["age"].quantile([0.25, 0.5, 0.75])
fig.add_trace(go.Bar(name="Возраст", x=df.index, y=df.values))
fig.add_vline(name="Медиана", legendrank=1, showlegend=True, x=median, line_color="red")
fig.add_vline(name="Мода", legendrank=1, showlegend=True,x=moda[0], line_color="purple")
fig.add_vline(name="Среднее значение", legendrank=1, showlegend=True,x=mean, line_color="orange")

fig.add_vline(name="Q1", legendrank=1, showlegend=True,x=quartiles[0.25], line_dash="dash", line_color="black", annotation_text="Q1")
fig.add_vline(name="Q2", legendrank=1, showlegend=True,x=quartiles[0.5], line_dash="dash", line_color="black", annotation_text="Q2")
fig.add_vline(name="Q3", legendrank=1, showlegend=True,x=quartiles[0.75], line_dash="dash", line_color="black", annotation_text="Q3")

fig.update_layout(
    title_text="Распределение возрастов",
    xaxis_title="Возраст",
    yaxis_title="Количество соискателей",
    annotations=[
        dict(
            xref="paper",
            yref="paper",
            x=.2,
            y=-.2,
            showarrow=False,
            text=f"""
      Медиана: {int(median*100)/100}
      Мода: {moda[0]}
      Среднее значение: {int(mean*100)/100}
      """,
            font=dict(size=16, color="grey"),
        )
    ])
fig.show()

36.0 37 37.21307506053269


In [136]:
resume_data["age"][resume_data["age"].between(31,41)].count()

np.int64(1945)

Резюме:
Аномалии, которые мы подозревали ранее отсутствуют.
Видим, что 2 и 3 квартили состоят из людей от 31 до 41 года (1945 человек).
Данные начиная с 55 лет малочисленны

В столце gender присутствуеют слова на 2 языках. Переведем с английского на русский.

In [137]:
resume_data["gender"] = [i.replace("Female", "Женщина") for i in resume_data["gender"]]
resume_data["gender"] = [i.replace("Male", "Мужчина") for i in resume_data["gender"]]
resume_data["gender"].unique()

array(['Мужчина', 'Женщина', 'Пол не указан'], dtype=object)

#### Resume_data. Создаем currency

In [138]:
curr = []
for i in resume_data["salary"]:
    temp = re.findall(r'руб|USD|EUR|RUB|KZT', i)
    curr.append(temp[0])
resume_data["currency"] = curr
resume_data["currency"] = [i.replace("руб", "RUB") for i in resume_data["currency"]]
resume_data.sample()

,index,title,salary,experience,gender,age,city,tags_list,tags_list_len,skills-content,language,work_text,specialization,currency
2874,2874,Коммерческий директор,400 000 руб.,Опыт работы 22 года 8 месяцев,Женщина,43,Москва,"['Продвижение бренда', 'Стратегический маркети...",10,Опыт работы с ПК: Уверенный пользователь - Win...,"['Русский — Родной', 'Английский — C2 — В сове...","Баркли\nМосква, barkli.ru\nСтроительство, недв...",Специализации:\nКоммерческий директор (CCO),RUB


#### Resume_data. Переопределим salary 

In [139]:
resume_data["salary"] = [int(i.replace('\u2009', '').split()[0]) for i in resume_data["salary"] ]

In [140]:
resume_data.loc[resume_data["salary"] > 10_000_000]

,index,title,salary,experience,gender,age,city,tags_list,tags_list_len,skills-content,language,work_text,specialization,currency
2643,2643,Руководитель отдела рекламы и PR,100000000,Обо мне,Женщина,21,Москва,[],0,В последнее время нахожусь в поисках работы.\nУ,"['Русский — Родной', 'Английский — C1 — Продви...",NaN,Специализации:\nРуководитель отдела маркетинга...,RUB
3041,3041,Инвестор в интернет-магазин женских аксессуаро...,30000000,Опыт работы 16 лет 10 месяцев,Женщина,39,Москва,"['Управленческий учет', 'Бюджетирование', 'Опт...",17,"Семейное положение: не замужем, детей нет\n\nХ...","['Русский — Родной', 'Английский — C1 — Продви...","Sorrisi\nМосква, www.sorrisimilano.com\nРуково...",Специализации:\nДиректор по информационным тех...,RUB


In [141]:
df = resume_data[resume_data["currency"]=="RUB"]
df = df.loc[df["salary"] > 0]
df["salary"].describe()

count    3.079000e+03
mean     2.962682e+05
std      1.885941e+06
min      8.000000e+04
25%      1.200000e+05
50%      1.800000e+05
75%      3.500000e+05
max      1.000000e+08
Name: salary, dtype: float64

Видим вакансии с зарплатами около 100 млн. рублей без опыта и навыков🤡
Смело удаляем такие вакансии из resume_data

In [142]:
resume_data = resume_data.loc[resume_data["salary"] <= 5_000_000]

Для удобства работы с experience, переведем года и месяцы опыта работы в число месяцев работы

In [143]:
curr = []
for i in resume_data["experience"]:
    temp= re.findall(r'\d+', i)
    temp = [int(i) for i in temp]
    if temp == []:
        temp.append(0)
    temp[0] *= 12
    curr.append(sum(temp))
resume_data["experience"] = curr
resume_data.sample()

,index,title,salary,experience,gender,age,city,tags_list,tags_list_len,skills-content,language,work_text,specialization,currency
605,605,Директор по интернет-маркетингу,150000,181,Женщина,34,Санкт-Петербург,"['SEO', 'SEO оптимизация', 'SEO-копирайтинг', ...",27,Только белая з/п!\nДополнительная информация:\...,"['Русский — Родной', 'Английский — B1 — Средни...",ООО Инпромконсалт\nСанкт-Петербург\nИнформацио...,Специализации:\nДиректор по маркетингу и PR (C...,RUB


В столбце language указывается список языков, разделенных запятыми, но в строковом формате. Исправим это

In [144]:
def str_to_list(str_repr):
  if str_repr == " ":
    return []
  else:
    return [s.strip().strip("'") for s in str_repr[1:-1].split(",")]
resume_data["tags_list"] = [str_to_list(i) for i in resume_data["tags_list"]]
resume_data["language"] = [str_to_list(i) for i in resume_data["language"]]
resume_data.sample()

,index,title,salary,experience,gender,age,city,tags_list,tags_list_len,skills-content,language,work_text,specialization,currency
1720,1720,Руководитель отдела маркетинга,180000,125,Мужчина,33,Москва,"[Планирование маркетинговых кампаний, Маркетин...",7,"Графика (Figma, Adobe Photoshop, CorelDRAW).\n...","[Русский — Родной, Английский — B1 — Средний]","Digital.tools\nМосква, digital.tools/\nКоммерч...",Специализации:\nДиректор по маркетингу и PR (C...,RUB


# Вывод

Переименовал столбцы в привычные для меня наименования. Большинство вакансий актуальны для Москвы, видим, что есть много пропусков в adress и city, предположим, что вакансии предполагают удаленку или находятся вне Москвы. В этих же столбцах есть дубликаты. Найдены аномальные данные, предполагаю, что некоторые вакансии и резюме фейковые.

Кратко:
* Переформатировал и разделил данные для удобной работы
* Найдены и удалены аномальные данные *(н-р люди с зарплатой ~10M руб. и 1-3 летним опытом)*
* Заполнил пропуски
* Рассмотрели нормальное распределение возрастов

# Анализируем данные

### Data. Какие требования и условия у работадателей?🧐

(1) Рассморим первое, что приходит в голову, злосчастный "Опыт работы". Ну и конечно нам интересна зарплатная вилка компаний

In [145]:
df = data[data["currency"]=="руб"]
df = df.loc[(df["from"] < 500_000) & (df["upto"] < 500_000)] 
fig = px.scatter(df, color="experience", x="from", y="upto")
fig.show()

В целом ожидаемо, видим, что с ростом опыта работы растет и размер заработной платы, однако видим также, что присутствуют странные вакансии предлагающие высокую зарплату при малом опыте и наоборот. Рассмотрим некоторую выборку из этих данных далее

In [146]:
df = data[data["experience"]=="не требуется"]
df = df[df["upto"]>400000]
df.sample(frac=0.5)

,index,experience,description,salary,tags_list,tags_list_len,from,upto,currency,fulladress
1633,1633,не требуется,Крупное агентство недвижимости «Владис» пригла...,от 100 000 до 500 000 руб. на руки,"[Жилая недвижимость, Ведение переговоров, Обуч...",12,100000.0,500000.0,руб,"Достоевская, Менделеевская, Новослободская, Мо..."
777,777,не требуется,MAD Estate\nСоздаем лучшие сделки с недвижимос...,от 150 000 до 500 000 руб. на руки,"[Ведение переговоров, Навыки продаж, Активные ...",17,150000.0,500000.0,руб,"Москва, улица Неверовского, 10с3А"


In [147]:
df = data[data["experience"]=="1–3 года"]
df = df[df["upto"]>400000]
df.sample(frac=0.5)

,index,experience,description,salary,tags_list,tags_list_len,from,upto,currency,fulladress
947,947,1–3 года,Madestate - эффективное агентство элитной недв...,от 200 000 до 600 000 руб. на руки,"[Элитная недвижимость, Жилая недвижимость, Вто...",15,200000.0,600000.0,руб,"Москва, улица Неверовского, 10с3А"
436,436,1–3 года,Мы - Университет Удаленных Профессий UNINEW. О...,от 100 000 до 500 000 руб. на руки,"[Анализ трафика, Ведение мобильных рекламных к...",10,100000.0,500000.0,руб,Москва
1619,1619,1–3 года,Обязанности:\nразрабатывать и реализовывать ст...,от 150 000 до 500 000 руб. на руки,"[арбитраж трафика, руководитель, affiliate mar...",6,150000.0,500000.0,руб,"деревня Новая Купавна, микрорайон Лагуна, 39/2"
430,430,1–3 года,Требуется Управляющий/Операционный директор в ...,от 400 000 до 600 000 руб. на руки,[],0,400000.0,600000.0,руб,"Домодедовская, Москва, Замоскворецкая линия, м..."


Резюме: 1 из 2, либо в сфере продаж недвижимости и wildberries действительно наблюдается тендеция к высоким зарплатам с минимумом опыта, либо, как в случае с некоторыми такими объявлениями остальные условия и требования обсуждаются индивидуально

(2) Второе на что хочется обратить внимание это теги, что за какие навыки работадатели готовы платить больше?

Число уникальных тегов в вакансиях:

In [148]:
df = data[data["tags_list_len"] > 0]["tags_list"]
tags_set = set()
for i in df:
    for j in i:
        tags_set.add(j)
len(tags_set)

1865

Число уникальных тегов в резюме:

In [149]:
df = resume_data[resume_data["tags_list_len"] > 0]["tags_list"]
tags_set = set()
for i in df:
    for j in i:
        tags_set.add(j)
len(tags_set)

4980

In [150]:
data.sample()

,index,experience,description,salary,tags_list,tags_list_len,from,upto,currency,fulladress
1010,1010,3–6 лет,Центр речевой неврологии Федора Краминова «Док...,от 90 000 руб. на руки,"[Управление персоналом, Организаторские навыки...",10,90000.0,0.0,руб,Москва


In [151]:
df = data[data["from"]<1_000_000]

fig = go.Figure()
fig.add_trace(go.Bar(x=df["tags_list_len"], y=df["upto"], name="upto"))
fig.add_trace(go.Bar(x=df["tags_list_len"], y=df["from"], name="from"))
fig.update_layout(
    title_text="Зависимость размера заработной платы от количества тегов",
    xaxis_title="Число тегов",
    yaxis_title="Зарплаты")
fig.show()


Видим, что соискатели указывают чаще всего от 4 до 10 тегов

In [152]:
df = data[data["from"]<5_000_000]
df = df.groupby("from").agg({'tags_list': 'count'})
df = df.sort_values(by="from").reset_index()
fig = go.Figure()
fig.add_trace(go.Bar(x=df["tags_list"], y=df["from"]))
fig.show()

In [153]:
data["tags_list_len"].unique()

array([ 5, 12,  9,  0, 28, 19, 13, 11,  8, 21,  7,  4,  6, 15, 10, 14, 26,
       24, 22,  3, 18, 20, 23, 16, 17,  2, 25, 29, 30,  1, 27, 31])

In [154]:

all_tags = []
for tags in data['tags_list']:
    for tag in tags:
        all_tags.append(tag)
len(set(all_tags))

1865

In [155]:
data.sample()

,index,experience,description,salary,tags_list,tags_list_len,from,upto,currency,fulladress
110,110,3–6 лет,"Производственно-торговая группа ""Aquanet"" - од...",от 150 000 руб. на руки,"[Маркетинговые исследования, Маркетинговые ком...",5,150000.0,0.0,руб,Москва


Нам очень интересна зависимость между зарплатой и необходимым стеком навыков. Построим график столбчатой диаграммы из средней начальной зарплаты, ее отклонения и уникальных тегов

In [156]:
df = data.explode('tags_list')

grouped_df = df.groupby(["tags_list"]).agg({"tags_list": "count"})
grouped_df.columns = ["count"]
grouped_df['from_mean'] = df.groupby('tags_list')['from'].apply(lambda x: x.sum()/len(x)).values
grouped_df['from_list'] = df.groupby('tags_list')['from'].apply(list).values
grouped_df['from_std'] = grouped_df['from_list'].apply(np.std)
print(f"Число уникальных тегов в data: {len(grouped_df)}")
grouped_df = grouped_df.reset_index().sort_values(by="from_mean")[-100:]
fig = go.Figure()
fig.add_trace(go.Bar(x=grouped_df["tags_list"], y=grouped_df["from_mean"]))
fig.add_trace(go.Scatter(x=grouped_df["tags_list"], y=grouped_df["from_std"]))
fig.show()

Число уникальных тегов в data: 1865


Рассмотрим интересный случай поближе

In [157]:
grouped_df = grouped_df.reset_index().sort_values(by="from_mean")[-20:]
fig = go.Figure()
fig.add_trace(go.Bar(x=grouped_df["tags_list"], y=grouped_df["from_mean"]))
fig.add_trace(go.Scatter(x=grouped_df["tags_list"], y=grouped_df["from_std"]))
fig.show()

Можем предположить, что навыки с большим отклонением средней заработной платы необходимы на разных уровнях должностей. Интересно, что самые "дорогие" теги связаны со сферами продаж, аналитики данных, менеджмента и управления

Число людей без опыта

In [158]:
df = data[data["experience"] == "не требуется"]
len(df)

151

Так как вакансий без тегов достаточно много, хочется рассмотреть и их

In [159]:
df = data[data["tags_list"].apply(lambda x: '' in x)]
print(df["experience"].value_counts().keys(), df["experience"].value_counts())
fig = go.Figure()
fig.add_trace(go.Histogram(name="С тегами", x=data["experience"]))
fig.add_trace(go.Bar(name="Без тегов",x=df["experience"].value_counts().keys(), y=df["experience"].value_counts()))
fig.show()

Index(['1–3 года', '3–6 лет', 'более 6 лет', 'не требуется'], dtype='object', name='experience') experience
1–3 года        63
3–6 лет         55
более 6 лет     22
не требуется    14
Name: count, dtype: int64


### Resume_data.

In [160]:
df = data.groupby("currency").agg({"currency": "count"})
df.columns = ["count"]
df.reset_index()

,currency,count
0,EUR,5
1,USD,28
2,не указана,1
3,руб,1673
4,сум,2


In [161]:
resume_data.sample()
df = resume_data.groupby(resume_data["title"]).agg({"title": "count"})
df.columns = ["count"]
df = df.sort_values(by="count", ascending=False).reset_index()
print(f"Число уникальных заголовков в resume_data: {len(df)}")
df = df[:20]
fig = go.Figure()
fig.add_trace(go.Bar(x=df["title"], y=df["count"]))
fig.show()

Число уникальных заголовков в resume_data: 1285


Видим, что значительное большинство соискателей нацелено на руководящие должности  и маркетинг

На всякий случай рассмотрим и независящий от соискателей gender


In [162]:

df = resume_data.loc[resume_data["salary"]<1000000]
df1 = df.loc[df["gender"]=="Мужчина"]
df2 = df.loc[df["gender"]=="Женщина"]
df1 = df1.groupby("age").agg({"age": "mean"})
df2 = df2.groupby("age").agg({"age": "mean"})

df["gender"] = resume_data["gender"]

fig = go.Figure()
fig.add_trace(go.Bar(name="Мужчины", x=df1.index, y=df1["age"]))
fig.add_trace(go.Bar(name="Женщины", x=df2.index, y=df2["age"]))
fig.show()

C:\Users\leedm\AppData\Local\Temp\ipykernel_22744\3655850233.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [163]:
resume_data.groupby(["gender"]).agg({"gender": "count"})

,gender
gender,
Женщина,1646
Мужчина,1653
Пол не указан,3


In [164]:
df = resume_data[resume_data["salary"]<1000000]

app = Dash("Salary")

dropdown = dcc.Dropdown([100_000, 200_000, 300_000, 400_000, 1_000_000], clearable=False)

graph = dcc.Graph()

app.layout = html.Div([html.H4("Зависимость размера заработной платы от пола"), dropdown, graph])

@callback(Output(graph, "figure"), Input(dropdown, "value"))
def update_bar_chart(salary):
    mask = df["salary"] < salary
    fig = px.bar(df[mask], x="age", y="salary", color="gender", barmode="group")
    return fig

app.run_server(debug=True)

In [165]:
resume_data[resume_data["salary"]<300000].groupby(["gender"]).agg({"gender": "count"})

,gender
gender,
Женщина,1305
Мужчина,1060
Пол не указан,3


На первый взгляд кажется, что при равенстве количества мужских и женских вакансий, женщины оценивают свою зарплату меньше, чем мужчины. Однако в срезе зарплат до 300_000 рублей мужчины оценивают свою зарплату меньше, чем женщины, правда это также в силу того, что в этом срезе количество женщин на 30% больше, чем мужчин.

Кроме того в пределах наших данных можно предположить, что в Москве женщины менее заинтересованны в поиске работы после 55 лет

# Вывод

### Данные и предобработка


### Вакансии
* Ожидаемо рост зарплаты напрямую зависит от опыта
* В вакансиях с тегами оптимизация логистических операций, Affiliate, стратегический менеджмент, медиабаинг наблюдается повышенное отклонение std по среднему значению начальной зарплаты. Предположительно так как эти навыки могут пригодится сотрудникам самых разных уровней
* С ростом зарплаты вакансии все чаще предполагают организационную деятельность

### Резюме
* Интересно, что большинство соискателей нацелено на руководящие должности и маркетинг
* Половина соискателей в возрасте от 32 до 41 года
* Равное количество мужчин и женщин
* В основном соискатели указывают чаще всего от 4 до 10 тегов
* Мужчины соискатели как правило оценивают свою зарплату больше, чем женщины
* Удалены вакансии с неестественными параметрами (например, вакансии с зарплатой большей 5 млн при низком опыте)

### Комментарий
Можно еще провести анализ соответствия тегов из resume_data с тегами из data. В выводе получим отношение ожидания работодателей к возможностям соискателей(однако мы уже видим, что работадатели ищут людей с тегами, примущим руководящим должностям, см. Вывод.Вакансии и Вывод.Резюме). Также с помощью geopy можно переформатировать адреса из city и city2 в координаты и составить гексогональную карту